In [20]:
!pip3 install -qU pybiolib
import biolib
import pandas as pd
deeptmhmm = biolib.load('DTU/DeepTMHMM')
biolib.login()


2023-06-05 20:46:16,667 | INFO : Loaded project DTU/DeepTMHMM:1.0.24
Opening authorization page at: https://biolib.com/sign-in/request/notebook/?token=1za86WTQbIi3nn8J
If your browser does not open automatically, click on the link above.


<IPython.core.display.Javascript object>

Successfully signed in!


2023-06-05 18:16:43,664 | INFO : Job "82df41eb-a3b0-4083-b253-c52750ab47c2" is starting...
2023-06-05 18:16:51,451 | INFO : Cloud: Initializing
2023-06-05 18:16:51,452 | INFO : Cloud: Pulling images...
2023-06-05 18:16:51,452 | INFO : Cloud: Computing...
Traceback (most recent call last):
  File "predict.py", line 41, in <module>
    fasta_size = os.stat(args.fasta).st_size
FileNotFoundError: [Errno 2] No such file or directory: 'query.fasta'
2023-06-05 18:16:54,014 | INFO : Cloud: Computation finished
2023-06-05 18:16:54,015 | INFO : Cloud: Result Ready


In [5]:
samtools = biolib.load('samtools/samtools')


data = pd.read_excel('Malaria_Research_Data.xlsx', header=0)

data = data['Peptide sequence'].head(1)

print(data)

results = []

for peptide in data:
    result = samtools.cli(peptide, blocking=True)
    print(result.get_stdout().decode("utf-8"))
    results.append(result)

# print(results[0].get_stdout().decode("utf-8"))
print("done!")
    

2023-06-05 19:21:57,355 | INFO : Loaded project samtools/samtools:0.0.79
0    EYILESGAVVLSDK
Name: Peptide sequence, dtype: object
2023-06-05 19:22:01,523 | INFO : Job "f9e9327b-1345-46e5-aff8-68e9e9c26d63" is starting...
2023-06-05 19:22:10,928 | INFO : Cloud: Initializing
2023-06-05 19:22:10,929 | INFO : Cloud: Downloading Source Files...
2023-06-05 19:22:10,929 | INFO : Cloud: Pulling images...
2023-06-05 19:22:10,929 | INFO : Cloud: Computing...
[main] unrecognized command 'EYILESGAVVLSDK'
2023-06-05 19:22:16,059 | INFO : Cloud: Computation finished
2023-06-05 19:22:16,060 | INFO : Cloud: Result Ready
[main] unrecognized command 'EYILESGAVVLSDK'

done!


In [24]:


deeptmhmm = biolib.load('DTU/DeepTMHMM')


def make_file(file_name, pep_seq):
    with open(file_name, "w") as file:
        file.write(pep_seq)

def write_results(file_name, output):
    with open(file_name, "wb") as file:
        file.write(output)


data = pd.read_excel('Malaria_Research_Data.xlsx', header=0)

data = data['Peptide sequence'].head(2)

print(data)

results = []

fasta_list = []

index = 0
for peptide in data:
    
    make_file("fasta_files/query"+str(index)+".fasta", peptide)
    fasta_list.append("fasta_files/query"+str(index)+".fasta")
    index+=1

print(fasta_list)
    
index = 0
with biolib.Experiment('TMHMM_analysis'):
    for fasta_file in fasta_list:
        result = deeptmhmm.cli(args='--fasta '+fasta_file)
        #results.append(result)
        result.save_files('transmembrane_results/peptide_row'+str(index))
        index+=1


'''
index = 0
for cloud_result in results:
    file_name = "transmembrane_results/"+str(index)+".txt"
    write_results(file_name, results[index].get_stdout())
    cloud_result.save_files('transmembrane_results/peptide_row'+str(index))
    index+=1
'''    
    
'''
print(results[0].get_stdout())
print(results[1].get_stdout())
'''

print("done!")

2023-06-05 21:05:49,344 | INFO : Loaded project DTU/DeepTMHMM:1.0.24
0       EYILESGAVVLSDK
1    LMNIATFQSLIDPLSGR
Name: Peptide sequence, dtype: object
['fasta_files/query0.fasta', 'fasta_files/query1.fasta']
2023-06-05 21:05:53,405 | INFO : Job "992d729e-c73d-4294-abea-ec88379f11a6" is starting...
2023-06-05 21:06:04,905 | INFO : Cloud: Initializing
2023-06-05 21:06:04,906 | INFO : Cloud: Pulling images...
2023-06-05 21:06:04,907 | INFO : Cloud: Computing...
Running DeepTMHMM on 1 sequence...
Step 1/4 | Loading transformer model...

Step 2/4 | Generating embeddings for sequences...
Generating embeddings: 100% 1/1 [00:00<00:00, 35.12seq/s]

Step 3/4 | Predicting topologies for sequences in batches of 1...
Topology prediction: 100% 1/1 [00:00<00:00, 21.51seq/s]

Step 4/4 | Generating output...
2023-06-05 21:06:27,341 | INFO : Cloud: Computation finished
2023-06-05 21:06:27,342 | INFO : Cloud: Result Ready
2023-06-05 21:06:32,869 | INFO : Saving 5 files to transmembrane_results/peptide_

In [26]:
samtools = biolib.load('samtools/samtools')

def make_file(file_name, pep_seq):
    with open(file_name, "w") as file:
        file.write(pep_seq)

def write_results(file_name, output):
    with open(file_name, "wb") as file:
        file.write(output)


data = pd.read_excel('output.xlsx', header=0)

data = data['Peptide sequence'].head(2)

parse_limit = 50

results = []
fasta_list = []

index = 0
for peptide in data:    
    make_file("fasta_files/query"+str(index)+".fasta", peptide)
    fasta_list.append("fasta_files/query"+str(index)+".fasta")
    index+=1


parsed_list = []
index = 0
while index < len(data):
    temp_list = []
    j=0
    while j < 50:
        temp_list.append(fasta_list[index])
        index+=1
        j+=1
    parsed_list.append(temp_list)
    

    
index = 0
with biolib.Experiment('TMHMM_analysis'):
    for fasta_file in fasta_list:
        result = samtools.cli(fasta_file, blocking=False)
        results.append(result)
        #result.save_files('transmembrane_results/peptide_row'+str(index))
        index+=1


'''
index = 0
for cloud_result in results:
    file_name = "transmembrane_results/"+str(index)+".txt"
    write_results(file_name, results[index].get_stdout())
    cloud_result.save_files('transmembrane_results/peptide_row'+str(index))
    index+=1
'''    
    
'''
print(results[0].get_stdout())
print(results[1].get_stdout())
'''

print("done!")

2023-06-05 21:11:52,373 | INFO : Loaded project samtools/samtools:0.0.79
0       EYILESGAVVLSDK
1    LMNIATFQSLIDPLSGR
Name: Peptide sequence, dtype: object
['fasta_files/query0.fasta', 'fasta_files/query1.fasta']
done!
